In [1]:
import pandas as pd
import geopandas as gpd
import sqlite3
from shapely.geometry import Point
import folium
from itertools import chain
import numpy as np
import markdown

In [2]:
db = sqlite3.connect("tests/sample_data/kapalo.sqlite")

In [3]:
observations = pd.read_sql_query("SELECT * from Observation;", db)
planar_structures = pd.read_sql_query("SELECT * from BFDS_Planar_structure;", db)
tectonic_measurements = pd.read_sql_query("SELECT * from Tectonic_measurement;", db)
rock_observations_points = pd.read_sql_query("SELECT * from Rock_observation_point;", db)
linear_structures = pd.read_sql_query("SELECT * from BFDS_Linear_structure;", db)
images = pd.read_sql_query("SELECT * from Outcrop_picture;", db)

In [4]:
{*observations.columns} & {*planar_structures.columns} & {*tectonic_measurements.columns}

{'GDB_ID', 'LAT', 'LON', 'OBJECTID'}

In [5]:
planar_merged = planar_structures.merge(observations, on="GDB_ID", how="left", suffixes=[None, "_y"], validate="one_to_one")
planar_merged = planar_merged.merge(rock_observations_points, on="GDB_ID", how="left", suffixes=[None, "_y"], validate="one_to_one")

In [6]:
planar_merged["geometry"] = [Point(x, y) for x, y in zip(planar_merged["LAT"], planar_merged["LON"])]

In [7]:
gdf_wgs84 = gpd.GeoDataFrame(planar_merged, crs="EPSG:4326")
gdf_tm35 = gdf_wgs84.to_crs("EPSG:3067")

In [8]:
def dip_colors(dip:float):
    color="blue"
    if dip > 60:
        color = "red"
    elif dip > 30:
        color = "green"
    return color

In [9]:
m = folium.Map(location=[gdf_wgs84["LAT"].median(), gdf_wgs84["LON"].median()], tiles="OpenStreetMap")
for idx, row in gdf_wgs84.iterrows():
    point = row["geometry"]
    dip = row["DIP"]
    dip_dir = row["DIRECTION_OF_DIP"]
    rotation = dip_dir - 90 if dip_dir > 90 else 270 + dip_dir
    rock_name = row["ROCK_NAME_TEXT"]
    measurement = f"{dip_dir:>3}/{dip:>2}".replace(" ", "0")
    folium.Marker(
        location=[point.x, point.y], 
        popup=folium.Popup(measurement, parse_html=True),
        icon=folium.Icon(angle=rotation, icon="glyphicon-chevron-right", icon_color=dip_colors(dip)),
        tooltip=str(dip)
    ).add_to(m)
m.save("index.html")

In [10]:
tectonic_observations = observations.merge(tectonic_measurements, on="OBSID", how="right", suffixes=[None, "_y"])

In [11]:
for col in ("GDB_ID", "BO_GID", "OBSID"):
    print(col)
    vals = planar_structures["TM_GID"].values
    other = tectonic_observations[col].values
    print(len(vals), len(other), len(set(chain(vals, other))))

GDB_ID
294 89 178
BO_GID
294 89 178
OBSID
294 89 178


In [12]:
planar_tectonic_obs = tectonic_observations.merge(planar_structures, left_on="GDB_ID_y", right_on="TM_GID", how="left")

In [13]:
linear_tectonic_obs = tectonic_observations.merge(linear_structures, left_on="GDB_ID_y", right_on="TM_GID", how="left")

In [14]:
linears = linear_tectonic_obs.loc[~np.isnan(linear_tectonic_obs["LAT_y"]).iloc[:,1]]

In [15]:
for col in tectonic_observations.columns:
    vals = tectonic_observations[col].values
    
    for second_col in linear_structures.columns:
        other_vals = linear_structures[second_col]
        
        for val in vals:
            
            for other_val in other_vals:
                
                if val == other_val:
                    if isinstance(val, str) and len(val) > 0:
                        print(col, second_col)
                        break

GDB_ID_y TM_GID
GDB_ID_y TM_GID
GDB_ID_y TM_GID
GDB_ID_y TM_GID
GDB_ID_y TM_GID
GDB_ID_y TM_GID
GDB_ID_y TM_GID
GDB_ID_y TM_GID
GDB_ID_y TM_GID


In [16]:
markdown_text = """
# Header

Body

-  Bullet

[![Image caption](piirros.png)](./piirros.png)
"""

In [17]:
markdown.markdown(markdown_text)

'<h1>Header</h1>\n<p>Body</p>\n<ul>\n<li>Bullet</li>\n</ul>\n<p><a href="./piirros.png"><img alt="Image caption" src="piirros.png" /></a></p>'

In [18]:
img_html = markdown.markdown(markdown_text).replace("src=", "width=500 src=")

In [19]:
m = folium.Map(location=[gdf_wgs84["LAT"].median(), gdf_wgs84["LON"].median()], tiles="OpenStreetMap")
folium.Marker(
    location=[gdf_wgs84["LAT"].median(), gdf_wgs84["LON"].median()],
    popup=folium.Popup(html=img_html, parse_html=False)
).add_to(m)

m.save("index.html")

In [20]:
planar_tectonic_obs["LAT_y"]

,LAT_y,LAT_y
0,60.370147,60.370147
1,60.357638,60.357638
2,60.357655,60.357655
3,60.358231,60.358231
4,60.363936,60.363936
...,...,...
289,60.416714,60.416714
290,60.416714,60.416714
291,60.416714,60.416714
292,60.416714,60.416714


In [24]:
tectonic_observations = observations.merge(tectonic_measurements, on="OBSID", how="right", suffixes=["_x", "_y"])